In [57]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\programdata\anaconda3\lib\site-packages (0.0)


In [58]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


from matplotlib import style
style.use("fivethirtyeight")

from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# Read the CSV and Perform Basic Data Cleaning

In [145]:
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [146]:
# Set features. This will also be used as your x values.
# test_features = df[['koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
# X = test_features

X = df.drop('koi_disposition', axis=1)
X = df[['koi_period']]
X = X.head(200)
# X = X.values.tolist()
X

# X = df.drop('koi_disposition', axis=1)
# X = df.head(200)
# X.head()

,koi_period
0,54.418383
1,19.899140
2,1.736952
3,2.525592
4,4.134435
...,...
195,42.140609
196,10.198493
197,5.795064
198,7.156816


# Create a Train Test Split

Use `koi_disposition` for the y values

In [170]:


# Assign koi_disposition to y
Y = df[['koi_disposition']]


# # Assign koi_disposition to y and Remove spaces btw words
# Y = df['koi_disposition'].apply(lambda str : str.replace(" ", ""))

# 200 samples for test
Y = Y.head(200)

# LABEL ENCODE Y
# Import required module
from sklearn.preprocessing import LabelEncoder

# LabelEncoder
le = LabelEncoder()

# Create an object of the label encoder class
labelencoder = LabelEncoder()

# apply "le.fit_transform"
y = Y.apply(le.fit_transform)
y

# Apply labelencoder object on columns
# labelencoder.fit_transform(data.ix[:, 1:])   # First column does not need to be encoded

,koi_disposition
0,1
1,2
2,2
3,1
4,1
...,...
195,1
196,1
197,2
198,1


In [148]:
# Split data into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [149]:
X_train

,koi_period
175,5.007317
132,10.443320
2,1.736952
50,7.240661
49,2.337438
...,...
65,0.444292
75,10.328036
108,4.144517
74,9.029307


In [150]:
y_train

,koi_disposition
175,1
132,2
2,2
50,0
49,1
...,...
65,2
75,2
108,1
74,0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [151]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

# Create variables to hold the scaled train & test data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [152]:
model1 = SVC(kernel="linear")
model1


# print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [165]:
# Create the GridSearchCV model
param_grid = {'C': [0.01, 1, 5],
              'gamma': [0.0005, 0.001, 0.005],
             'n_splits_': [2]}
kf = KFold(n_splits=2, shuffle=False, random_state=None)
newxtrain=kf.split(np.array(X_train['koi_period']))
#grid = GridSearchCV(model1, param_grid, verbose=3, scoring='f1', cv=cv)

In [169]:
for train_index , test_index in kf.split(X_train):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

KeyError: "None of [Int64Index([38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,\n            55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,\n            72, 73, 74],\n           dtype='int64')] are in the [columns]"

In [155]:
newxtrain=np.array(X_train['koi_period'])
newxtrain

array([5.00731666e+00, 1.04433199e+01, 1.73695245e+00, 7.24066120e+00,
       2.33743795e+00, 1.44964519e+01, 2.53918315e+00, 2.99025443e+00,
       1.20398749e+01, 2.00901704e+01, 1.27180987e+01, 1.03716893e+01,
       7.95354700e+00, 6.48163307e+00, 1.62663022e+00, 1.01984925e+01,
       1.96512095e+01, 1.32335221e+01, 4.11972550e+01, 2.68889923e+00,
       1.15214461e+01, 1.63297527e+01, 1.15230675e+01, 6.12563628e+01,
       1.00065301e+01, 2.04774884e+01, 2.05234988e+00, 5.16980427e+00,
       3.13304614e+01, 3.64454001e+01, 1.53756237e+01, 3.43591631e+00,
       9.43945949e+00, 2.81650490e+00, 3.94105221e+00, 3.30655164e+00,
       4.09357846e+00, 4.58352095e+00, 3.39337345e+01, 3.04033046e+00,
       2.18129600e+01, 3.36012337e+01, 3.18002790e+00, 1.26910439e+01,
       1.47491482e+01, 5.90224578e+00, 5.44183827e+01, 3.60658816e+01,
       9.75192101e+00, 2.24338057e+00, 3.33612311e+00, 1.38102032e+01,
       7.11373257e+00, 2.79541337e+01, 6.03249422e+01, 4.60181182e+00,
      

In [156]:
newytrain=np.array(y_train['koi_disposition'])

In [123]:
# Train the model with GridSearch
grid.fit(newxtrain, newytrain)

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
# List best parameters and best score
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)